# Generate aggregated time series for Rinne daily data

This script generates different aggregations of select time series. Aggregations are done with a backwards-looking moving window. We can't use the standard .rolling function in pandas because it will not ignore nans. 

First, import the libraries we need:

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import copy
from scipy.signal import lfilter, butter
import matplotlib.image as mpimg
from matplotlib import rcParams
import pylab
import importlib

# Local modules
import TEpython3 


Now import the datafile from RinneData_QC:

In [2]:
table = pd.read_csv('Cleaned_Daily.csv',header = 0,index_col = 'Year_Month_Day',parse_dates = True, infer_datetime_format = True,low_memory=False)

print(table.shape)
table.head(5)

(3652, 15)


,T_air,Rh,R,T_-5cm,T_-10cm,T_-20cm,T_-35cm,T_-50cm,WT,NEE,Reco,GPP,methane_flux,P,H2O
Year_Month_Day,,,,,,,,,,,,,,,
2005-01-01,-1.39,89.8,18.60,-0.04,-0.4,-0.2,0.5,1.30,9.16,0.21,0.21,0.0,NaN,0.4,-0.060365
2005-01-02,-0.30,98.4,4.74,0.06,-0.4,-0.2,0.5,1.30,9.07,0.21,0.21,0.0,NaN,4.7,NaN
2005-01-03,-0.84,98.2,11.70,0.06,-0.4,-0.2,0.5,1.30,9.03,0.27,0.27,0.0,NaN,0.6,NaN
2005-01-04,-2.58,99.9,11.20,0.04,-0.4,-0.2,0.5,1.30,8.66,0.26,0.26,0.0,NaN,3.3,NaN
2005-01-05,-2.76,100.0,7.75,0.02,-0.4,-0.2,0.5,1.28,8.81,0.23,0.23,0.0,NaN,7.0,NaN


Here is the plan for a rolling mean that ignores NaNs.
 - Save an ~isnan array (1 for valid datapoint, 0 for nan datapoint)
 - Replace nans with zeros
 - Compute a rolling sum over the nan-replaced datacolumn
 - Compute a rolling sum over the ~isnan array
 - Divide the first rolling sum above by the second.

In [3]:
def nanrollingmean(df, win): #df = input dataframe, win = window of rolling mean (backwards-looking)
    valid_entries = ~np.isnan(df.copy(deep=True))
    not_valid = np.isnan(df.copy(deep=True))
    df = df.fillna(0) #Replace nans with zeros
    rolling_mean = df.rolling(window=win).sum()/valid_entries.rolling(window=win).sum()
    rolling_mean[~np.isfinite(rolling_mean)] = np.nan #Replace divide-by-zeros with nan
    rolling_mean[not_valid]=np.nan
    return rolling_mean

### Develop table of aggregations for day-of-year anomaly analysis

In [4]:
#Define desired aggregation windows and variables to aggregate
data4DOY = copy.deepcopy(table)
agg_scales = np.array([7, 14, 30, 91, 183, 365])
vars_to_agg = np.array(['T_air', 'Rh', 'R', 'T_-5cm', 'T_-10cm', 'WT', 'NEE', 'Reco', 'GPP', 'P', 'H2O'])

#Append columns of aggregated data
for ii in vars_to_agg:
    for jj in agg_scales:
        data4DOY['{}_{}_day'.format(ii,jj)] = nanrollingmean(data4DOY[ii], jj)
data4DOY.head(10)        


,T_air,Rh,R,T_-5cm,T_-10cm,T_-20cm,T_-35cm,T_-50cm,WT,NEE,...,P_30_day,P_91_day,P_183_day,P_365_day,H2O_7_day,H2O_14_day,H2O_30_day,H2O_91_day,H2O_183_day,H2O_365_day
Year_Month_Day,,,,,,,,,,,,,,,,,,,,,
2005-01-01,-1.39,89.8,18.60,-0.04,-0.40,-0.2,0.5,1.30,9.16,0.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-02,-0.30,98.4,4.74,0.06,-0.40,-0.2,0.5,1.30,9.07,0.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-03,-0.84,98.2,11.70,0.06,-0.40,-0.2,0.5,1.30,9.03,0.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-04,-2.58,99.9,11.20,0.04,-0.40,-0.2,0.5,1.30,8.66,0.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,-2.76,100.0,7.75,0.02,-0.40,-0.2,0.5,1.28,8.81,0.23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,-1.64,100.0,9.35,-0.01,-0.40,-0.2,0.5,1.21,8.71,0.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,0.83,100.0,6.97,0.05,-0.40,-0.2,0.5,1.20,9.15,0.19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-08,1.28,95.9,13.60,0.09,-0.40,-0.2,0.5,1.20,10.70,0.27,...,NaN,NaN,NaN,NaN,-0.082281,NaN,NaN,NaN,NaN,NaN
2005-01-09,-0.12,99.8,7.71,0.10,-0.40,-0.2,0.5,1.20,11.70,0.31,...,NaN,NaN,NaN,NaN,0.014180,NaN,NaN,NaN,NaN,NaN


### Develop table of aggregations for moving average anomaly analysis

In [5]:
#Define desired aggregation windows and variables to aggregate
data4MA = copy.deepcopy(table)
agg_scales = np.array([2, 3, 4, 5, 6, 7])
vars_to_agg = np.array(['T_air', 'Rh', 'R', 'T_-5cm', 'T_-10cm', 'WT', 'NEE', 'Reco', 'GPP', 'P', 'H2O'])

#Append columns of aggregated data
for ii in vars_to_agg:
    for jj in agg_scales:
        data4MA['{}_{}_day'.format(ii,jj)] = nanrollingmean(data4MA[ii], jj)
data4MA.head(10)        


,T_air,Rh,R,T_-5cm,T_-10cm,T_-20cm,T_-35cm,T_-50cm,WT,NEE,...,P_4_day,P_5_day,P_6_day,P_7_day,H2O_2_day,H2O_3_day,H2O_4_day,H2O_5_day,H2O_6_day,H2O_7_day
Year_Month_Day,,,,,,,,,,,,,,,,,,,,,
2005-01-01,-1.39,89.8,18.60,-0.04,-0.40,-0.2,0.5,1.30,9.16,0.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-02,-0.30,98.4,4.74,0.06,-0.40,-0.2,0.5,1.30,9.07,0.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-03,-0.84,98.2,11.70,0.06,-0.40,-0.2,0.5,1.30,9.03,0.27,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-04,-2.58,99.9,11.20,0.04,-0.40,-0.2,0.5,1.30,8.66,0.26,...,2.250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,-2.76,100.0,7.75,0.02,-0.40,-0.2,0.5,1.28,8.81,0.23,...,3.900,3.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,-1.64,100.0,9.35,-0.01,-0.40,-0.2,0.5,1.21,8.71,0.20,...,3.625,3.84,3.266667,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,0.83,100.0,6.97,0.05,-0.40,-0.2,0.5,1.20,9.15,0.19,...,5.250,4.32,4.383333,3.814286,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-08,1.28,95.9,13.60,0.09,-0.40,-0.2,0.5,1.20,10.70,0.27,...,6.950,6.22,5.283333,5.200000,-0.082281,-0.082281,-0.082281,-0.082281,-0.082281,-0.082281
2005-01-09,-0.12,99.8,7.71,0.10,-0.40,-0.2,0.5,1.20,11.70,0.31,...,7.150,7.12,6.483333,5.642857,0.014180,0.014180,0.014180,0.014180,0.014180,0.014180


### Save outputs

In [6]:
data4DOY.to_csv('Aggregated_Daily_for_DOY.csv', sep=',')
data4MA.to_csv('Aggregated_Daily_for_MA.csv', sep=',')